# Preprocessing des données

Importation des dépendances

In [1]:
import pandas as pd
pd.options.plotting.backend = "plotly"

import numpy as np
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler


Chargement des notebooks et concaténations dans un unique DataFrame

In [2]:
test = pd.read_csv("X_test_Hi5.csv", low_memory=False, index_col="row_index")
train = pd.read_csv("X_train_Hi5.csv", low_memory=False, index_col="row_index")
full = pd.concat([train, test])

## Sélection et Imputing

Suppression des colonnes non utilisées et traitement du jour (ajout de 3 colonnes année, mois, jour)

In [3]:
drop_cols = ["piezo_station_department_code",
             "piezo_station_update_date",
             "piezo_station_department_name",
             "piezo_station_commune_code_insee",
             "piezo_station_pe_label",
             "piezo_station_bdlisa_codes",
             "piezo_station_bss_code",
             "piezo_station_commune_name",
             #"piezo_station_bss_id", #for groupby
             "piezo_bss_code",
             "piezo_obtention_mode",
             "piezo_continuity_code",
             "piezo_continuity_name",
             "piezo_producer_code",
             "piezo_producer_name",
             "piezo_measure_nature_name",
             "meteo_id",
             "meteo_name",
             "meteo_date",
             "meteo_DRR",
             "meteo_time_tn",
             "meteo_time_tx",
             "meteo_temperature_avg_threshold",
             "meteo_temperature_min_ground",
             "meteo_temperature_min_50cm",
             "meteo_frost_duration",
             "meteo_amplitude_tn_tx",
             "meteo_temperature_avg_tntm",
             "meteo_pressure_avg",
             "meteo_pression_maxi",
             "meteo_wind_speed_avg_10m",
             "meteo_wind_max",
             "meteo_wind_direction_max_inst",
             "meteo_time_wind_max",
             "meteo_wind_avg",
             "meteo_wind_direction_max_avg",
             "meteo_time_wind_avg",
             "meteo_wind_speed_avg_2m",
             "meteo_wind_max_2m",
             "meteo_wind_direction_max_inst_2m",
             "meteo_time_wind_max_2m",
             "meteo_wind_max_3s",
             "meteo_time_wind_max_3s",
             "meteo_time_humidity_min",
             "meteo_time_humidity_max",
             "meteo__pressure_saturation_avg",
             "meteo_wetting_duration",
             "meteo_sunshine_duration",
             "meteo_radiation",
             "meteo_radiation_direct",
             "meteo_sunshine_%",
             "meteo_radiation_IR",
             "meteo_radiation_UV_max",
             "meteo_cloudiness",
             "meteo_cloudiness_height",
             "meteo_if_snow",
             "meteo_if_fog",
             "meteo_if_thunderstorm",
             "meteo_if_sleet",
             "meteo_if_hail",
             "meteo_if_dew",
             "meteo_if_black_ice",
             "meteo_if_snow_ground",
             "meteo_if_frost",
             "meteo_if_smoke",
             "meteo_if_mist",
             "meteo_if_lightning",
             "meteo_evapotranspiration_Monteith",
             "meteo_radiation_UV",
             "meteo_snow_height",
             "meteo_snow_thickness_max",
             "meteo_snow_thickness_6h",
             "distance_piezo_meteo",
             "hydro_station_code",
             "hydro_observation_date_elab",
             "hydro_status_label",
             "hydro_method_code",
             "hydro_method_label",
             "hydro_qualification_label",
             "hydro_hydro_quantity_elab",
             "distance_piezo_hydro",
             "prelev_structure_code_0",
             "prelev_volume_0",
             "prelev_structure_code_0",
             "prelev_volume_0",
             "prelev_usage_label_0",
             "prelev_volume_obtention_mode_label_0",
             "prelev_longitude_0",
             "prelev_latitude_0",
             "prelev_commune_code_insee_0",
             "prelev_structure_code_1",
             "prelev_volume_1",
             "prelev_usage_label_1",
             "prelev_volume_obtention_mode_label_1",
             "prelev_longitude_1",
             "prelev_latitude_1",
             "prelev_commune_code_insee_1",
             "prelev_structure_code_2",
             "prelev_volume_2",
             "prelev_usage_label_2",
             "prelev_volume_obtention_mode_label_2",
             "prelev_longitude_2",
             "prelev_latitude_2",
             "prelev_commune_code_insee_2",
             "prelev_other_volume_sum",
             "insee_pop_commune",
             "insee_med_living_level",
             "insee_%_ind",
             "insee_%_const"
             ]

df = full.drop(columns=drop_cols)

df['piezo_measurement_date'] = pd.to_datetime(df['piezo_measurement_date'], format='mixed')
df['year'] = df['piezo_measurement_date'].dt.year
df['month'] = df['piezo_measurement_date'].dt.month
df['day'] = df['piezo_measurement_date'].dt.day

Imputing simple pour les valeurs manquantes

In [4]:
df["insee_%_agri"] = pd.to_numeric(df["insee_%_agri"], errors="coerce")

for f in ["piezo_station_investigation_depth", "meteo_evapotranspiration_grid", "insee_%_agri"]:
    df[f] = SimpleImputer(strategy="median").fit_transform(df[[f]])

df["piezo_measure_nature_code"] = df["piezo_measure_nature_code"].fillna('0')
df["meteo_rain_height"] = df["meteo_rain_height"].fillna(0)

df = pd.get_dummies(df, columns = ["piezo_status", "piezo_qualification", "piezo_measure_nature_code", "hydro_status_code", "hydro_qualification_code"], drop_first=True)

Imputing par un gradient boost multi-cible pour les données météo (température et humidité)

In [5]:
def impute_linked_values(df, train_cols, target_cols):
    """
    Impute les valeurs manquantes de colonnes liées en utilisant une régression multi-cible.

    Args:
        df (pd.DataFrame): Le DataFrame contenant les colonnes avec des valeurs manquantes.
        target_cols (list): Liste de deux colonnes liées à imputer ensemble.
    """
    predict_rows = df[df[target_cols].isna().any(axis=1)]
    train_rows = df[~df[target_cols].isna().any(axis=1)]
    X_train = train_rows[train_cols]
    y_train = train_rows[target_cols]
    X_predict = predict_rows[train_cols]
    model = MultiOutputRegressor(xgb.XGBRegressor(random_state=42))
    model.fit(X_train, y_train)
    y_pred = model.predict(X_predict)
    df.loc[predict_rows.index, target_cols] = y_pred

# On sépare en deux appels différents car les données supprimées (lignes où les Y sont manquants) ne sont pas les mêmes

impute_linked_values(df, ["year", "month", "day", "meteo_latitude", "meteo_longitude", "meteo_altitude"], ["meteo_temperature_min", "meteo_temperature_max", "meteo_temperature_avg"])

impute_linked_values(df, ["year", "month", "day", "meteo_latitude", "meteo_longitude", "meteo_altitude"], ["meteo_humidity_min", "meteo_humidity_max", "meteo_humidity_avg", "meteo_humidity_duration_below_40%", "meteo_humidity_duration_above_80%"])

## Augmentation

Coordonnées des points représentant la cyclicité des nappes et entraînement d'un kNN

In [6]:
arr_pluriannuel = [
    (49.288099, -0.203341), (47.879094, 1.687775), (49.175556, 2.233091),
    (50.262098, 3.073414), (48.923617, 3.931616), (45.081144, 5.465724),
    (46.004560, 6.295893), (46.477399, 5.205279)
]

arr_annuel = [
    (48.371338, -3.666128), (48.511718, -0.687289), (46.879448, -1.012845),
    (44.084124, -0.654733), (44.503562, 3.414720), (46.320239, 2.226440),
    (47.674577, 4.293722), (48.855616, 5.482002), (49.631358, 4.244888)
]

arr_mixte = [
    (50.473620, 1.867051), (48.845081, 7.399492), (47.988020, 6.505182),
    (46.556602, 3.851500), (45.053291, 3.260751), (49.301387, -0.583806),
    (47.529302, 0.845920), (48.633676, 2.874028)
]

arr_sans_grande_nappe = [
    (50.987253, 2.447802), (46.109438, 6.516357), (44.858971, 6.446409),
    (42.106361, 9.156892), (41.480571, 9.139405), (42.636047, 9.086944),
    (48.633676, 2.874028), (43.156810, 0.899236), (42.950893, 2.579219),
    (43.786061, 0.693303)
]


df_pluriannuel = pd.DataFrame(arr_pluriannuel, columns=["latitude", "longitude"])
df_pluriannuel["category"] = "pluriannuel"

df_annuel = pd.DataFrame(arr_annuel, columns=["latitude", "longitude"])
df_annuel["category"] = "annuel"

df_mixte = pd.DataFrame(arr_mixte, columns=["latitude", "longitude"])
df_mixte["category"] = "mixte"

df_sans_grande_nappe = pd.DataFrame(arr_sans_grande_nappe, columns=["latitude", "longitude"])
df_sans_grande_nappe["category"] = "sans_grande_nappe"

df_knn = pd.concat([df_pluriannuel, df_annuel, df_mixte, df_sans_grande_nappe],ignore_index=True)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(df_knn[["latitude", "longitude"]], df_knn["category"])

KNeighborsClassifier()

Insertion du cycle dans chaque ligne du jeu initial et one-hot encodage

In [8]:
unique_stations = df.drop_duplicates(subset="piezo_station_bss_id")
stations_coord = unique_stations[["piezo_station_bss_id", "piezo_station_latitude", "piezo_station_longitude"]].rename(columns={
    "piezo_station_latitude": "latitude",
    "piezo_station_longitude": "longitude"
})
stations_coord["aquifer_cyclicity"] = knn.predict(stations_coord[["latitude", "longitude"]])
stations_coord = pd.get_dummies(stations_coord[["piezo_station_bss_id", "aquifer_cyclicity"]], columns=["aquifer_cyclicity"], drop_first=True)
df = df.reset_index().merge(stations_coord, on='piezo_station_bss_id', how='left', left_index=False).set_index("row_index")

Ajout des features temporelles, notamment d'une fonction sinus qui faut -1 le 15 septembre et +1 le 15 mars (remplissage des nappes)
et de l'indicatrice de la période qui manque

In [10]:
df['doy'] = df['piezo_measurement_date'].dt.dayofyear
df['year_sinus'] = np.sin(2 * np.pi * (df['doy'] - 74) / 365)
df['summer'] = df['month'].isin([6, 7, 8, 9])

## Normalisation

Normalisation de chaque feature

In [11]:
for f in ["piezo_station_investigation_depth", "meteo_altitude","meteo_rain_height","insee_%_agri"]:
    df[f] = np.log(1 + df[f])

df["meteo_humidty_max"] = np.log(107 - df["meteo_humidity_max"])

for f in ["piezo_station_longitude", "piezo_station_latitude", "meteo_latitude", "meteo_longitude", "meteo_temperature_min", "meteo_temperature_max", "meteo_temperature_avg", "meteo_humidity_min", "meteo_humidity_avg", "meteo_evapotranspiration_grid", "hydro_longitude", "hydro_latitude"]:
    df[f] = StandardScaler().fit_transform(df[[f]])

for f in ["piezo_station_investigation_depth", "piezo_station_altitude", "meteo_altitude", "meteo_rain_height", "meteo_humidity_max", "meteo_humidity_duration_below_40%", "meteo_humidity_duration_above_80%", "hydro_observation_result_elab", "insee_%_agri"]:
    df[f] = RobustScaler().fit_transform(df[[f]])

## Lagging (sliding window) -- PAS UTILISE

Lagging des variables intéressantes sur 30 jours

In [ ]:
df_filtered = df.copy()
df_filtered = df_filtered.sort_values(['piezo_station_bss_id', 'piezo_measurement_date'])
df_filtered = df_filtered.reset_index(drop=False)
num_lags = 30
lag_variables = ['meteo_rain_height', 'meteo_temperature_avg', 'meteo_evapotranspiration_grid', 'hydro_observation_result_elab']

grouped = df_filtered.groupby('piezo_station_bss_id')

for var in lag_variables:
    for lag in range(1, num_lags + 1):
        lag_col = f'{var}_lag_{lag}'
        df_filtered[lag_col] = grouped[var].shift(lag)
        df_filtered[lag_col] = grouped[lag_col].ffill()
        df_filtered[lag_col] = df_filtered[lag_col].fillna(df_filtered[var])


## Export

Export en csv

In [12]:
df.to_hdf("X_preprocessed_final.h5", key="full", mode="w")

C:\Users\Concordance\AppData\Local\Temp\ipykernel_27244\4158562811.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block4_values] [items->Index(['piezo_station_bss_id', 'piezo_groundwater_level_category'], dtype='object')]

  df.to_hdf("X_preprocessed_final.h5", key="full", mode="w")
